# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [9]:
import numpy
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to maxdwyman@gmail.com and will expire on September 17, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1542643637.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.818214     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.875886     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [59]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    nfeatures = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned fit for number of features ' + str(nfeatures) + ':'
    print(w)

print((model_all.coefficients['value']).nnz())
coeffs = model_all.coefficients['value']
nonzeros = [coeffs > 0.1]
feats_nz = nonzeros[1:]
feats = np.asarray(all_features)
print(feats[np.asarray(coeffs[1:]) != 0])
print(feats[np.asarray(coeffs[:17]) != 0])

6
['bathrooms' 'sqft_living' 'sqft_living_sqrt' 'grade' 'sqft_above']
['bedrooms' 'sqft_living' 'sqft_living_sqrt' 'sqft_lot' 'sqft_above'
 'sqft_basement']


In [58]:
print(len(nonzeros),len(feats))
print([np.asarray(coeffs) != 0])
print_coefficients(model_all)
print(feats[np.asarray(coeffs[:17]) != 0])

(1, 17)
[array([ True, False, False,  True,  True,  True, False, False, False,
       False, False, False, False,  True,  True, False, False, False])]
Learned fit for number of features 17:
[274873.0559504957, 0.0, 0.0, 8468.531086910072, 24.420720982445214, 350.0605533860648, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 842.0680348976282, 20.024722417091304, 0.0, 0.0, 0.0]
['bedrooms' 'sqft_living' 'sqft_living_sqrt' 'sqft_lot' 'sqft_above'
 'sqft_basement']


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [12]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [64]:
?training

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [78]:
import numpy as np

def lasso_regression(data, l1_penalty = 0):
    model = graphlab.linear_regression.create(data, target='price', l2_penalty=0., features = all_features,
                                              l1_penalty=l1_penalty,
                                              validation_set=None, 
                                              verbose=False)
    return model

In [82]:
for l1_penalty in np.logspace(1, 7, num=13):
    model = lasso_regression(training, l1_penalty=l1_penalty)
    print 'lambda = %.2e' % l1_penalty
    print sum((validation['price']-model.predict(validation))**2)
    print((model.coefficients['value']).nnz())
    print '\n'
#    plt.figure()
#    plot_predictions(training,model)
#    plt.title('Ridge, lambda = %.2e' % l1_penalty)

lambda = 1.00e+01
6.25766285142e+14
18


lambda = 3.16e+01
6.25766285362e+14
18


lambda = 1.00e+02
6.25766286058e+14
18


lambda = 3.16e+02
6.25766288257e+14
18


lambda = 1.00e+03
6.25766295212e+14
18


lambda = 3.16e+03
6.25766317206e+14
18


lambda = 1.00e+04
6.25766386761e+14
18


lambda = 3.16e+04
6.25766606749e+14
18


lambda = 1.00e+05
6.25767302792e+14
18


lambda = 3.16e+05
6.25769507644e+14
18


lambda = 1.00e+06
6.25776517727e+14
18


lambda = 3.16e+06
6.25799062845e+14
18


lambda = 1.00e+07
6.25883719085e+14
18




In [74]:
training

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1.0,0
9212900260,2014-05-27 00:00:00+00:00,468000.0,2.0,1.0,1160.0,6000,1.0,0
0114101516,2014-05-28 00:00:00+00:00,310000.0,3.0,1.0,1430.0,19901,1.5,0
6054650070,2014-10-07 00:00:00+00:00,400000.0,3.0,1.75,1370.0,9680,1.0,0
9297300055,2015-01-24 00:00:00+00:00,650000.0,4.0,3.0,2950.0,5000,2.0,0
6865200140,2014-05-29 00:00:00+00:00,485000.0,4.0,1.0,1600.0,4300,1.5,0
6300500875,2014-05-14 00:00:00+00:00,385000.0,4.0,1.75,1620.0,4980,1.0,0
7137970340,2014-07-03 00:00:00+00:00,285000.0,5.0,2.5,2270.0,6300,2.0,0
8091400200,2014-05-16 00:00:00+00:00,252700.0,2.0,1.5,1070.0,9643,1.0,0


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [86]:
for l1_penalty in np.logspace(9.5, 10, num=11):
    model = lasso_regression(training, l1_penalty=l1_penalty)
    print 'lambda = %.2e' % l1_penalty
    print sum((validation['price']-model.predict(validation))**2)
    print((model.coefficients['value']).nnz())
    print '\n'

lambda = 3.16e+09
1.00194263832e+15
10


lambda = 3.55e+09
1.05747532365e+15
7


lambda = 3.98e+09
1.10251387902e+15
6


lambda = 4.47e+09
1.17744103152e+15
6


lambda = 5.01e+09
1.27111458564e+15
5


lambda = 5.62e+09
1.35290989011e+15
3


lambda = 6.31e+09
1.40167459821e+15
3


lambda = 7.08e+09
1.24066226108e+15
1


lambda = 7.94e+09
1.23018635848e+15
1


lambda = 8.91e+09
1.23025718345e+15
1


lambda = 1.00e+10
1.22915716064e+15
1




In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [ ]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [96]:
for l1_penalty_values in np.logspace(8, 10, num=20):
    model = lasso_regression(training, l1_penalty=l1_penalty_values)
    print 'lambda = %.5e' % l1_penalty_values
    print sum((validation['price']-model.predict(validation))**2)
    print((model.coefficients['value']).nnz())
    print '\n'

lambda = 1.00000e+08
6.27492659875e+14
18


lambda = 1.27427e+08
6.28210516771e+14
18


lambda = 1.62378e+08
6.29176689541e+14
18


lambda = 2.06914e+08
6.30650082719e+14
18


lambda = 2.63665e+08
6.32940229287e+14
17


lambda = 3.35982e+08
6.3626814023e+14
17


lambda = 4.28133e+08
6.41261198311e+14
17


lambda = 5.45559e+08
6.48983455376e+14
17


lambda = 6.95193e+08
6.60962217696e+14
17


lambda = 8.85867e+08
6.77261520728e+14
16


lambda = 1.12884e+09
7.01046815867e+14
15


lambda = 1.43845e+09
7.37850622829e+14
15


lambda = 1.83298e+09
7.9616310964e+14
13


lambda = 2.33572e+09
8.69018172894e+14
12


lambda = 2.97635e+09
9.66925692362e+14
10


lambda = 3.79269e+09
1.08186759232e+15
6


lambda = 4.83293e+09
1.24492736032e+15
5


lambda = 6.15848e+09
1.38416149024e+15
3


lambda = 7.84760e+09
1.23079472046e+15
1


lambda = 1.00000e+10
1.22915716064e+15
1




Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [91]:
l1_penalty_min = 2.98e+09
l1_penalty_max = 3.79e+09

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [98]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [99]:
for l1_penalty in l1_penalty_values:
    model = lasso_regression(training, l1_penalty=l1_penalty)
    print 'lambda = %.5e' % l1_penalty
    print sum((validation['price']-model.predict(validation))**2)
    print((model.coefficients['value']).nnz())
    coeffs = model.coefficients['value']
    feats = np.asarray(all_features)
    print(feats[np.asarray(coeffs[1:]) != 0])
    print(feats[np.asarray(coeffs[:17]) != 0])
    print '\n'

lambda = 2.98000e+09
9.67525133002e+14
10
['bedrooms' 'bathrooms' 'sqft_living' 'sqft_living_sqrt' 'floors' 'view'
 'grade' 'sqft_above' 'sqft_basement']
['bedrooms' 'bedrooms_square' 'sqft_living' 'sqft_living_sqrt' 'sqft_lot'
 'floors_square' 'condition' 'sqft_above' 'sqft_basement' 'yr_built']


lambda = 3.02263e+09
9.74569913357e+14
10
['bedrooms' 'bathrooms' 'sqft_living' 'sqft_living_sqrt' 'floors' 'view'
 'grade' 'sqft_above' 'sqft_basement']
['bedrooms' 'bedrooms_square' 'sqft_living' 'sqft_living_sqrt' 'sqft_lot'
 'floors_square' 'condition' 'sqft_above' 'sqft_basement' 'yr_built']


lambda = 3.06526e+09
9.81688588145e+14
10
['bedrooms' 'bathrooms' 'sqft_living' 'sqft_living_sqrt' 'floors' 'view'
 'grade' 'sqft_above' 'sqft_basement']
['bedrooms' 'bedrooms_square' 'sqft_living' 'sqft_living_sqrt' 'sqft_lot'
 'floors_square' 'condition' 'sqft_above' 'sqft_basement' 'yr_built']


lambda = 3.10789e+09
9.89838262746e+14
10
['bedrooms' 'bathrooms' 'sqft_living' 'sqft_living_sqrt' '

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?